In [ ]:
import polars as pl
from time import perf_counter

t_start = perf_counter()
# Read file as string
data: str
with open('samples.json', 'r') as file:
    data = file.read()

t2 = perf_counter()

result = pl.LazyFrame({"arrs": data}).select(pl.col("arrs").str.json_extract(pl.List(pl.List(pl.UInt8))).explode()).with_columns(
    id= pl.int_range(0, pl.col('arrs').count())
).explode("arrs").group_by('id').agg(pl.col('arrs').value_counts(parallel=True)).explode("arrs").unnest('arrs').group_by('id').agg([
    pl.col('arrs').filter(pl.col('counts').mod(2) == 1)
]).explode('arrs').collect().get_column('arrs').to_list()

# Stop the timer
t3 = perf_counter()

print(result)

t_stop = perf_counter()

print(f"File reading {round((t2-t_start) * 1000, 2)}ms")
print(f"Compute {round((t3-t2) * 1000, 2)}ms")
print(f"Print {round((t_stop-t3) * 1000, 2)}ms")


In [ ]:
data: str
with open('samples.json', 'r') as file:
    data = file.read()

In [ ]:
parsed_json = pl.DataFrame({"arrs": data}).select(pl.col("arrs").str.json_extract(pl.List(pl.List(pl.UInt8))).explode())
print(parsed_json)

In [ ]:
ids = parsed_json.with_columns(
    id= pl.int_range(0, pl.col('arrs').count())
)
print(ids)

In [ ]:
counts = ids.explode("arrs").group_by('id').agg(pl.col('arrs').value_counts(parallel=True)).explode("arrs").unnest('arrs')
print(counts)

In [ ]:
find_odds = counts.group_by('id').agg([pl.col('arrs').filter(pl.col('counts').mod(2) == 1)])
print(find_odds)

In [ ]:
result = find_odds.explode('arrs').get_column('arrs').to_list()
print(result)

In [ ]:
import polars as pl
from time import perf_counter

t_start = perf_counter()

with open("samples.json") as f:
   df = pl.select(arrs = pl.lit(f.read()).str.json_extract().explode())
   
selected = (df.with_row_count("id")
   .explode("arrs")
   .group_by("id", "arrs", maintain_order=True)
   .count()
   .filter(pl.col("count").mod(2) == 1)
   .select("arrs")
).get_column('arrs').to_list()

print(selected, sep='')

In [ ]:
with open("samples.json") as f:
   print((pl.LazyFrame(["arrs", pl.List(pl.Utf8)]).select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode()).with_row_count("id").explode("arrs").group_by("id", "arrs", maintain_order=True).count().filter(pl.col("count").mod(2) == 1).collect().get_column('arrs').to_list()), sep='')

In [ ]:
from time import perf_counter

def get_odds():
    t_start = perf_counter()

    with open("samples.json") as f:
        pl.LazyFrame(["arrs", pl.List(pl.Utf8)]).select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode()).lazy().with_row_count("id").explode("arrs").group_by("id", "arrs", maintain_order=True).count().filter(pl.col("count").mod(2) == 1).collect().get_column('arrs').to_list()

    t_stop = perf_counter()

    return t_stop-t_start

times = [get_odds() for i in range(1000)]
round(sum(times) / len(times) * 1000, 2)

In [57]:
from time import perf_counter

# https://stackoverflow.com/questions/77257229/find-int-in-list-with-odd-count-with-polars-dataframe/77259576#77259576

def wth_rw_cnt(file_path):
    with open(file_path) as f:
        return (
            pl.LazyFrame(["arrs", pl.List(pl.Utf8)]).select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode())
            .with_row_count("id").explode("arrs")
            .group_by("id", "arrs", maintain_order=True).count()
            .filter(pl.col("count").mod(2) == 1)
            .collect().get_column("arrs").to_list()
        )

def lst_vl(file_path):
    with open(file_path) as f:
        return (
            pl.LazyFrame(["arrs", pl.List(pl.Utf8)])
            .select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode())
            .select(pl.col("arrs").list.eval(pl.element().value_counts()))
            .explode("arrs")
            .filter(pl.col("arrs").struct["counts"].mod(2) == 1)
            .select(arrs = pl.col("arrs").struct[""])
            .collect()
            .get_column("arrs")
            .to_list()
        )

def lst_fltr(file_path):
    with open(file_path) as f:
        return (
            pl.LazyFrame(["arrs", pl.List(pl.Utf8)]).select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode())
            .select(
                pl.col("arrs") 
                    .list.eval(pl.element().value_counts())
                    .list.eval(
                    pl.element().filter(pl.element().struct["counts"].mod(2) == 1)
                        .struct[""]
                    )
                    .flatten()
            ).collect().get_column("arrs").to_list()
        )

for algo in [wth_rw_cnt, lst_vl, lst_fltr]:
    time_sum = 0
    filepath = "samples.json"
    iters = 1000
    for i in range(iters):
        t_start = perf_counter()
        algo(filepath)
        t_end = perf_counter()
        time_sum += (t_end - t_start)
    print(round(time_sum / iters * 1000, 2))

7.83
5.98
6.69


In [ ]:
with open("samples.json") as f:
    df = pl.select(arrs = pl.lit(f.read()).str.json_extract(pl.List(pl.List(pl.UInt8))).explode())

In [ ]:
(df.lazy().select(pl.col("arrs").list.eval(pl.element().value_counts()))
   .explode("arrs")
   .filter(pl.col("arrs").struct["counts"].mod(2) == 1)
   .select(arrs = pl.col("arrs").struct[""])
   .collect()
   .get_column("arrs")
   .to_list()
)

In [39]:
df.lazy().select(
   pl.col("arrs") 
     .list.eval(pl.element().value_counts())
     .list.eval(
        pl.element().filter(pl.element().struct["counts"].mod(2) == 1)
          .struct[""]
     )
     .flatten()
).collect().get_column("arrs").to_list()

[5,
 85,
 51,
 43,
 98,
 42,
 36,
 64,
 39,
 52,
 88,
 11,
 62,
 23,
 19,
 99,
 68,
 83,
 34,
 35,
 70,
 35,
 35,
 85,
 42,
 54,
 89,
 64,
 85,
 11,
 92,
 15,
 7,
 12,
 84,
 6,
 65,
 70,
 6,
 83,
 91,
 7,
 13,
 87,
 50,
 35,
 37,
 66,
 39,
 99,
 81,
 17,
 75,
 24,
 69,
 20,
 38,
 97,
 79,
 87,
 99,
 16,
 51,
 82,
 40,
 14,
 95,
 99,
 54,
 14,
 88,
 94,
 84,
 18,
 66,
 48,
 18,
 41,
 58,
 39,
 16,
 99,
 76,
 16,
 91,
 95,
 48,
 7,
 22,
 29,
 8,
 62,
 63,
 50,
 50,
 64,
 72,
 95,
 57,
 78,
 99,
 31,
 99,
 97,
 81,
 96,
 11,
 28,
 62,
 36,
 43,
 76,
 3,
 44,
 27,
 21,
 53,
 61,
 78,
 40,
 85,
 68,
 89,
 94,
 13,
 17,
 43,
 40,
 42,
 86,
 4,
 39,
 23,
 32,
 4,
 59,
 22,
 70,
 35,
 83,
 15,
 10,
 10,
 33,
 15,
 89,
 58,
 99,
 53,
 89,
 73,
 62,
 81,
 72,
 8,
 63,
 28,
 42,
 4,
 13,
 46,
 42,
 90,
 42,
 93,
 7,
 38,
 7,
 33,
 1,
 38,
 94,
 68,
 77,
 98,
 46,
 73,
 90,
 9,
 88,
 66,
 11,
 76,
 40,
 86,
 66,
 31,
 31,
 10,
 34,
 58,
 55,
 64,
 71,
 72,
 10,
 89,
 80,
 49,
 65,
 63,
 58,
 87,
 

In [ ]:
import polars as pl

In [ ]:
with open("samples.json") as f:
    result: list[int] = pl.LazyFrame(["arrs", pl.List(pl.Utf8)]).select(arrs = pl.lit(f.read()).str.json_extract().explode()).with_row_count("id").explode("arrs").group_by("id", "arrs", maintain_order=True).count().filter(pl.col("count").mod(2) == 1).select("arrs").collect().get_column('arrs').to_list()

In [ ]:
print(result, sep=",")

In [ ]:
with open("samples.json") as f:
        print(pl.select(arrs = pl.lit(f.read()).str.json_extract().explode()).lazy().with_row_count("id").explode("arrs").group_by("id", "arrs", maintain_order=True).count().filter(pl.col("count").mod(2) == 1).select("arrs").collect().get_column('arrs'))


In [ ]:
selected